In [2]:
from neuroposelib import read
from neuroposelib import vis
import numpy as np
import time
from IPython.display import Video
from pathlib import Path
import matplotlib.pyplot as plt
# %matplotlib inline
from neuroposelib import preprocess
from neuroposelib import write
from neuroposelib import features
from neuroposelib import DataStruct as ds
from neuroposelib.embed import Embed
from neuroposelib import analysis
import pandas as pd
from neuroposelib.embed import Watershed

config = read.config('/home/lq53/mir_repos/BBOP/random tests/dappy_backups/iter_com_features/mir_1.yaml')

connectivity = read.connectivity(
    path=config["skeleton_path"], skeleton_name=config["skeleton_name"]
)

# Make out_path
Path(config["out_path"]).mkdir(parents=True, exist_ok=True)

# meta, meta_by_frame = read.meta(config["data_path"] + "demo_meta.csv", id=ids)
pose, ids, meta, meta_by_frame = read.pose_from_meta(
    path=config["meta_path"], connectivity=connectivity, key="Prediction_path", file_type="dannce"
)

Path(config["out_path"]).mkdir(parents=True, exist_ok=True)

# write.pose_h5(pose,ids, config['out_path'] + 'pose_merged_newcol.h5')


pose = preprocess.rotate_spine(preprocess.center_spine(pose, keypt_idx=4), keypt_idx=[4, 3])


rel_vel, rel_vel_labels = features.get_velocities(
    pose,
    ids,
    connectivity.joint_names,
    joints=np.delete(np.arange(18), 4),
    widths=[5, 11, 51],
    f_s = 30
)

angles, angle_labels = features.get_angles(pose, connectivity.angles)

ego_pose, labels = features.get_ego_pose(pose, connectivity.joint_names)
# # Ensure all feature datasets have the same number of samples

# Angular Velocities
angular_vel, angular_vel_labels = features.get_angular_vel(
        angles,
        angle_labels,
        ids,
        widths=[5, 11, 51],
    )
# write.features_h5(
#     angular_vel, angular_vel_labels, path=config["out_path"] + "angular_velocity.h5"
# )

# Compute and save Euler angles (if applicable)
euler_angles, euler_labels = features.get_euler_angles(pose, connectivity.angles)
# write.features_h5(
#     euler_angles, euler_labels, path=config["out_path"] + "euler_angles.h5"
# )

# Compute and save head angular velocities
head_angular = features.get_head_angular(pose, ids, widths=[5, 10, 50])
# write.features_h5(
#     head_angular, ["head_angular"], path=config["out_path"] + "head_angular.h5"
# )


115it [00:06, 17.41it/s]


Centering poses to mid spine ...
Rotating spine to xz plane ... 
Detected centered pose input - calculating relative velocities ... 


100%|██████████| 115/115 [01:18<00:00,  1.47it/s]


Calculating joint angles ... 


100%|██████████| 55/55 [00:04<00:00, 11.53it/s]


Reformatting pose to egocentric pose features ... 


In [6]:
config = read.config('/home/lq53/mir_repos/dappy_24_nov/byws_version/mir_1.yaml')

In [13]:
config["out_path"]

'/home/lq53/mir_repos/dappy_24_nov/byws_version/250115pti_comb_all/50_p/'

In [17]:
# Make out_path
Path(config["out_path"]).mkdir(parents=True, exist_ok=True)

In [3]:
# # Angular Velocities
# angular_vel, angular_vel_labels = features.get_angular_vel(
#         angles,
#         angle_labels,
#         ids,
#         widths=[5, 11, 51],
#     )
# # write.features_h5(
# #     angular_vel, angular_vel_labels, path=config["out_path"] + "angular_velocity.h5"
# # )

# # Compute and save Euler angles (if applicable)
# euler_angles, euler_labels = features.get_euler_angles(pose, connectivity.angles)
# # write.features_h5(
# #     euler_angles, euler_labels, path=config["out_path"] + "euler_angles.h5"
# # )

# # Compute and save head angular velocities
# head_angular = features.get_head_angular(pose, ids, widths=[5, 10, 50])
# # write.features_h5(
# #     head_angular, ["head_angular"], path=config["out_path"] + "head_angular.h5"
# # )


Calculating velocities of angles ... 


100%|██████████| 115/115 [00:51<00:00,  2.23it/s]


Calculating joint angles - Euler ... 


  4%|▎         | 2/55 [00:01<00:34,  1.53it/s]/home/lq53/mir_repos/dappy_24_nov/neuroposelib/src/neuroposelib/features.py:165: RuntimeWarning: invalid value encountered in divide
  v2_u = v2[:, plane_dict[key]] / np.expand_dims(
  9%|▉         | 5/55 [00:02<00:14,  3.36it/s]/home/lq53/mir_repos/dappy_24_nov/neuroposelib/src/neuroposelib/features.py:162: RuntimeWarning: invalid value encountered in divide
  v1_u = v1[:, plane_dict[key]] / np.expand_dims(
100%|██████████| 115/115 [00:01<00:00, 70.48it/s]


In [18]:
# # assert rel_vel.shape[0] == angles.shape[0] == ego_pose.shape[0]

# # # Combine features
# # combined_features = np.hstack([rel_vel, angles, ego_pose])

# # # Concatenate labels
# # combined_labels = rel_vel_labels + angle_labels + labels

# assert head_angular.shape[0] == euler_angles.shape[0] == angular_vel.shape[0] == rel_vel.shape[0]

# # Combine all features
# combined_features = np.hstack([rel_vel, angles, ego_pose, head_angular, euler_angles, angular_vel])
# combined_features = np.nan_to_num(combined_features, nan=0.0, posinf=0.0, neginf=0.0)

# # Combine all labels
# combined_labels = rel_vel_labels + angle_labels + labels + ["head_angular"] + euler_labels + angular_vel_labels



# # Perform PCA on the combined features
# pca_features, pca_labels = pca(
#     features=combined_features,
#     labels=combined_labels,
#     categories=["vel", "ang", "ego_pose", "head_ang", "eluer_ang", "avel"], #["vel", "ego_euc", "ang", "avel"],
#     n_pcs=10,
#     downsample=1,
#     method="fbpca",
# )

write.features_h5(pca_features, pca_labels, path=config["out_path"] + "_combined_pca.h5")


pc_feats, pc_labels = pca_features, pca_labels

wlet_feats, wlet_labels = features.wavelet(
    pc_feats, pc_labels, ids, f_s=30, freq=np.linspace(1, 25, 25), w0=5
)

write.features_h5(wlet_feats, wlet_labels, path=config["out_path"] + "_wavelets.h5")

# PCA on wavelet features
pc_wlet, pc_wlet_labels = pca(
    wlet_feats,
    wlet_labels,
    # categories=["wlet_ego_euc"],
    categories=["wlet_ang"],
    n_pcs=5,
    method="fbpca",
)

del wlet_feats, wlet_labels
pc_feats = np.hstack((pc_feats, pc_wlet))
pc_labels += pc_wlet_labels
del pc_wlet, pc_wlet_labels

write.features_h5(
    pc_feats, pc_labels, path="".join([config["out_path"], "_pca_on_wavelets.h5"])
)




data_obj = ds.DataStruct(
    pose=pose,
    id=ids,
    meta=meta,
    meta_by_frame=meta_by_frame,
    connectivity=connectivity,
)

data_obj.features = pc_feats
# When using high framerate data, downsampling may be necessary in order to 
# discover granular structure in embedding
data_obj = data_obj[:: config["downsample"], :]


embedder = Embed(
    embed_method=config["single_embed"]["method"],
    perplexity=config["single_embed"]["perplexity"],
    lr=config["single_embed"]["lr"],
)
data_obj.embed_vals = embedder.embed(data_obj.features, save_self=True)

# Watershed clustering
data_obj.ws = Watershed(
    sigma=config["single_embed"]["sigma"], max_clip=1, log_out=True, pad_factor=0.05
)
data_obj.data["Cluster"] = data_obj.ws.fit_predict(data=data_obj.embed_vals)
print("Writing Data Object to pickle")
data_obj.write_pickle(''.join([config['out_path'],'/']))



freq, combined_keys = analysis.cluster_freq_by_cat(
                        data_obj.data["Cluster"].values, cat=data_obj.id
                    )
freq_df = pd.DataFrame(freq.T, columns=combined_keys)
freq_df.to_csv(''.join([config['out_path'],'/cluster_occupancy.csv']))

vis.pose.sample_grid3D(
    pose-pose.mean(axis=-2, keepdims=True),
    connectivity=connectivity,
    
    labels=data_obj.data["Cluster"],
    n_samples=9,
    centered=True,
    N_FRAMES=100,
    fps=30,
    dpi=100,
    watershed=data_obj.ws,
    embed_vals=None,
    VID_NAME = "cluster",
    filepath=config["out_path"],
)


Calculating wavelets ... 
Calculating wavelets for video 0.0


100%|██████████| 60/60 [00:02<00:00, 26.38it/s]


Calculating wavelets for video 1.0


100%|██████████| 60/60 [00:02<00:00, 26.57it/s]


Calculating wavelets for video 2.0


...

 98%|█████████▊| 115/117 [1:05:02<01:10, 35.21s/it]

[2066320, 2067940, 2077720, 2073730, 2063850, 2097580, 2112320, 2060270, 2098370]


 99%|█████████▉| 116/117 [1:05:37<00:35, 35.27s/it]

[2076890, 2093900, 2107510, 2102610, 2107880, 2088940, 2102330, 2064100, 2088590]


100%|██████████| 117/117 [1:06:12<00:00, 33.96s/it]


In [12]:
# nan_indices = np.isnan(combined_features)
# inf_indices = np.isinf(combined_features)
# print("NaN values at:", np.argwhere(nan_indices))
# print("Inf values at:", np.argwhere(inf_indices))
combined_features = np.nan_to_num(combined_features, nan=0.0, posinf=0.0, neginf=0.0)


In [15]:
from tqdm import tqdm
from typing import Optional, Union, List, Tuple, Type

def pca(
    features: np.ndarray,
    labels: List,
    categories: List[str] = ["vel", "ego_euc", "ang", "avel"],
    n_pcs: int = 10,
    downsample: int = 1,
    method="fbpca",
):
    print("Calculating principal components ... ")

    # Initializing the PCA method
    # if method.startswith("torch"):
    #     import torch

    #     pca_feats = torch.zeros(features.shape[0], len(categories) * n_pcs)
    #     features = torch.tensor(features)
    # else:
    # Centering the features if not torch (pytorch does it itself)
    features = features - features.mean(axis=0)
    pca_feats = np.zeros(
        (features.shape[0], len(categories) * n_pcs), dtype=features.dtype
    )

    if method == "ipca":
        from sklearn.decomposition import IncrementalPCA

        pca = IncrementalPCA(n_components=n_pcs, batch_size=None)
    elif method.startswith("fbpca"):
        import fbpca

    num_cols = 0
    for i, cat in enumerate(tqdm(categories)):  # Iterate through each feature category
        cat += "_"
    
        cols_idx = [
            i
            for i, col in enumerate(labels)
            if (col.startswith(cat) or ("_" + cat in col))
            # if (col.decode('utf-8').startswith(cat) or ("_" + cat in col.decode('utf-8')))

        ]
        num_cols += len(cols_idx)
        # print(f"Category: {cat}, Matching labels: {[label for label in combined_labels if cat in label]}")
        if not cols_idx:
            print(f"Skipping category {cat}: no matching columns found.")
            continue

        if method == "ipca" or method == "sklearn_pca":
            # import pdb; pdb.set_trace()
            pca_feats[:, i * n_pcs : (i + 1) * n_pcs] = pca.fit_transform(
                features[:, cols_idx]
            )

        # elif method.startswith("torch"):
        #     feat_cat = features[:, cols_idx]
        #     if method.endswith("_gpu"):
        #         feat_cat = feat_cat.cuda()

        #     if "pca" in method:
        #         (_, _, V) = torch.pca_lowrank(feat_cat)
        #     elif "svd" in method:
        #         feat_cat -= feat_cat.mean()
        #         (_, _, V) = torch.linalg.svd(feat_cat)

        #     if method.endswith("_gpu"):
        #         pca_feats[:, i * n_pcs : (i + 1) * n_pcs] = (
        #             torch.matmul(feat_cat, V[:, :n_pcs]).detach().cpu()
        #         )
        #         feat_cat.detach().cpu()
        #         V.detach().cpu()
        #     else:
        #         pca_feats[:, i * n_pcs : (i + 1) * n_pcs] = torch.matmul(
        #             feat_cat, V[:, :n_pcs]
        #         )

        elif method == "fbpca":
            print(f"Category: {cat}, Columns: {cols_idx}, Feature shape: {features[:, cols_idx].shape}")

            (_, _, V) = fbpca.pca(
                features[::downsample, cols_idx].astype(np.float64), k=n_pcs
            )
            pca_feats[:, i * n_pcs : (i + 1) * n_pcs] = np.matmul(
                features[:, cols_idx], V.astype(features.dtype).T
            )

    # if method.startswith("torch_pca"):
    #     pca_feats = pca_feats.numpy()

    # assert num_cols == features.shape[1]

    pc_labels = [
        "_".join([cat, "pc" + str(i)]) for cat in categories for i in range(n_pcs)
    ]

    return pca_feats, pc_labels